In [1]:
import sys
sys.path.append('/Users/exrhizo/Projects/graphistry/pygraphistry')

In [2]:

import numpy as np
import graphistry
#from graphistry.compute.chat_utils import *
from graphistry.compute.ai.symbolic import *

import pandas as pd
import os

graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username=os.environ['USERNAME'], password=os.environ['GRAPHISTRY_PASSWORD']) 

In [3]:
query = 'write a splunk query for the index `redteam_50k` that uses the src and dst information \
to output a table for events where red=1'

In [4]:
query2 = 'write a splunk query that uses the src and dst information \
to output a table for events and \
include any fields similar to time and place if they exist in the data'

In [5]:
query3 = 'write a splunk query that uses the src and dst information \
to output a table for events where auth_type is equal to Kerberos'

In [6]:
g = graphistry.bind()
# query to graph 
g2 = g.ai(query)

In [25]:
g2

<class 'symai.symbol.Expression'>(value=index=redteam_50k red=1 | table src, dst)

In [24]:
g2.plot()  # all red events

AttributeError: 'str' object has no attribute 'plot'

In [ ]:
## query 2

In [26]:
g.ai(query2).plot()

AttributeError: 'str' object has no attribute 'plot'

In [ ]:
#g.ai(query3).plot()

## Check to see that it understands the difference between splunk and normal queries

In [8]:
g3 = g.ai('What is the capital of Norway?')

In [9]:
g3

<class 'symai.symbol.Expression'>(value=Oslo)

## Getting into the details

In [10]:
sym = SplunkAIGraph('redteam_50k')
sym

<class 'graphistry.compute.ai.symbolic.SplunkAIGraph'>(value=None)

In [18]:
sym.connect("exrhizo", "XdC*-Hex.jdNBX4UH83p", "splunk.graphistry.com")

100.0%   19762 scanned   19762 matched   28 results
Done!
len results 28
Generated splunk context


In [19]:
sym.splunk_search(query)

100.0%   749 scanned   749 matched   749 results
Done!
len results 749
context ***Found Data:
  src_computer dst_computer timestamp
0       C19932         C108      none
1       C17693         C828      none
2       C17693         C828      none
3       C17693         C828      none
4       C17693         C828      none
------------------------------
--Added a successful memory: write a splunk query for the index `redteam_50k` that uses the src and dst information to output a table for events where red=1


,src_computer,dst_computer,timestamp
0,C19932,C108,none
1,C17693,C828,none
2,C17693,C828,none
3,C17693,C828,none
4,C17693,C828,none
...,...,...,...
744,C17693,C294,none
745,C17693,C1173,none
746,C17693,C728,none
747,C17693,C305,none


In [ ]:
sym.get_context('trump', all_indexes=True)

In [20]:
sym._splunk_context

"You are working with the index `redteam_50k` and the following fields: ['RED', 'auth_type', 'authentication_orientation', 'dst_computer', 'dst_domain', 'eventtype', 'extracted_index', 'feats', 'feats2', 'host', 'index', 'linecount', 'logontype', 'node', 'punct', 'source', 'sourcetype', 'splunk_server', 'splunk_server_group', 'src_computer', 'src_domain', 'success_or_failure', 'tag', 'tag::eventtype', 'time', 'timestamp', 'unix_category', 'unix_group'] You may also use the following indexes: dict_keys(['_audit', '_internal', '_introspection', '_telemetry', '_thefishbucket', 'accenture', 'accenture-labs', 'achp', 'alert_graph_demo', 'anons_bad_day', 'anonsbadday', 'arcsight2', 'arcsightcsv', 'attck', 'auth1m', 'bahamas_papers', 'bitcointransaction', 'cim_modactions', 'cisco', 'corelight_tutorial', 'creditone', 'cylance_test', 'deletemedemo', 'dvd', 'dvd_debug', 'dvd_fraud', 'dvd_fraud_tp', 'dvdtmx', 'event_gen', 'fct-demo', 'femsa1', 'findsearch', 'gen', 'gen_anon', 'graphistry_apps', '

In [ ]:
sym._static_context

In [ ]:
sym.splunk._static_context

In [ ]:
sym.index

In [ ]:
list(sym.indexes.keys())

# Add a straight splunk query
It will reason about the input, as shown below

In [27]:
gg = sym.splunk_search('| search index="redteam_50k" | Table src_computer, dst_computer')

HTTP 400 Bad Request -- Unknown search command 'action'.
context !!The following SPL query returned no results: action=login | Where RED=1
Empty DataFrame
Columns: []
Index: []
100.0%   0 scanned   0 matched   0 results
Done!
----------------------------------------
new splunk, who dis?
	 search index="redteam_50k" RED=1 auth_type=login | Table src_computer, dst_computer, success_or_failure, timestamp 

context !!The following SPL query returned no results: search index="redteam_50k" RED=1 auth_type=login | Table src_computer, dst_computer, success_or_failure, timestamp
Empty DataFrame
Columns: []
Index: []
100.0%   0 scanned   0 matched   0 results
Done!
----------------------------------------
new splunk, who dis?
	 search index="redteam_50k" RED=1 auth_type=login success_or_failure!="" | Table src_computer, dst_computer, success_or_failure, timestamp 

context !!The following SPL query returned no results: search index="redteam_50k" RED=1 auth_type=login success_or_failure!="" | Tab

In [28]:
gg

""


In [ ]:
sym.fields

In [ ]:
#q2 = 'search index="redteam_50k" src_computer=* OR dst_computer=* | table src_computer dst_computer eventtype success_or_failure'

In [14]:
from graphistry.compute.ai.ai_prompts import Splunk

In [15]:
splunk = Splunk()

In [22]:
splunk.query('write a splunk query to detect anomalous winlogs events')

<class 'symai.symbol.Symbol'>(value=index=wineventlog EventID=4624 OR EventID=4625 | eval count_events=1 | stats sum(count_events) as total_events | eval anomaly_threshold=total_events*0.01 | eval anomaly_threshold_lower=total_events-anomaly_threshold | eval anomaly_threshold_upper=total_events+anomaly_threshold | where count_events<anomaly_threshold_lower OR count_events>anomaly_threshold_upper | table _time, EventID, count_events, anomaly_threshold)